In [1]:
import numpy as np
from pyscf import gto,scf
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf

### Morse potential
The Morse potential is the best fit for the binding potential of a diatomic molecule. <br>
It's parameters are: $D_e$ , the positive Dissociation Energy, $r_e$ the distance of equilibrium, and $a$ a parameter that gives the width of the energy hole.<br>
$a$ is correlated to the second derivative $k$ in the minum bt the relationship:
$ a = \sqrt{k_e/2D_e} $ and at reverse $ k_e = 2D_e a^2 $ <br>
If we assume the energy in the minimum to be zero we use the following expression:
$$ V_{Morse}(r)= D_e (1-e^{-a(r-r_e)})^2
$$
If other wise we chose to fit the total elettrostatic plus kinetic energy of the system, than we can express the potential as :
$$  V_{Morse}(r)= D_e (e^{-a2(r-r_e)}-2e^{-a(r-r_e)})+V(r=\infty)
$$
Such expression can be usefull because the V at infinite can be estimate accuratly from QM calculation for separated atoms (as long as the method chosen is size-consistent) , once for all. <br>
what we are left with is an expression of the potential in terms of only three parameters. <br>
If a method posses analytical first and second derivative we can fit with one single scf the whole potential energy curve of the biatomic dissociation. <br>
Such a scf can also come from an alchemycal perturbed density functional calculation... <br>
If this thing will in future be implemented, would be usefull the expression for the first and second derivatives of the Morse potential: 
$$  V_{Morse}(r)= D_e (e^{-2a(r-r_e)}-2e^{-a(r-r_e)})+D_e
$$ <br>
$$ \frac{dV_{Morse}}{dr}= -2a D_e(e^{-2a(r-r_e)}-e^{-a(r-r_e)}) 
$$ <br>
$$ \frac{dV_{Morse}^2}{dr^2}= -2a^2 D_e(e^{-a(r-r_e)}-2e^{-2a(r-r_e)})
$$ 
if $t=e^{-a(r-r_e)}$

$$ \frac{dV_{Morse}}{dr}= 2D_e a (t-t^2) 
$$ <br>
$$ \frac{dV_{Morse}^2}{dr^2}= 2 D_e a^2 (2t^2-t)
$$ 

$$ \tilde g = \frac{g}{2D_e} $$<br>
$$\tilde h = \frac{h}{2D_e}$$

$$ \tilde{g}= a t(1-t) 
$$ <br>
$$  \tilde h = a^2 t(2t-1)
$$ <br>
$$ \frac{\tilde g ^2}{\tilde h} =Z= \frac{ t(1-t)^2}{2t-1}
$$

$$ t^3 -2t^2+t =Z(2t-1) $$ <br>$$
t^3-2t^2+(1-2Z)t-Z=0
$$
To be solved for t>0

In [2]:
def MorseV(r,D,a,re):
    return D*(np.exp(-2*a*(r-re))-2*np.exp(-a*(r-re)))+D

In [3]:
np.roots([2,1,2,1])

array([ 2.91433544e-16+1.j,  2.91433544e-16-1.j, -5.00000000e-01+0.j])

In [4]:
mol=gto.M(atom="N 0 0 0;N 0 0 2.1427", unit="Bohr")
mf=scf.RHF(mol)
mf.scf()
mf.Gradients().grad()[1,2]

converged SCF energy = -107.500654261793
--------------- RHF gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000000000    -0.0000601570
1 N     0.0000000000     0.0000000000     0.0000601570
----------------------------------------------


6.015699931083418e-05

In [39]:
mol=gto.M(atom="C 0 0 0;O 0 0 2.1646", unit="Bohr")
mf=scf.RHF(mol)
mf.scf()
mf.Gradients().grad()[1,2]

converged SCF energy = -111.225449512559
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.0000706232
1 O    -0.0000000000    -0.0000000000    -0.0000706232
----------------------------------------------


-7.062321091133583e-05

In [27]:
R0=1.95
mol=gto.M(atom="C 0 0 0;O 0 0 {}".format(R0), unit="Bohr")
mf=scf.RHF(mol)
e_co=mf.scf()
g=mf.Gradients().grad()[1,2]
h=mf.Hessian().hess()[1,1,2,2]

converged SCF energy = -111.179064277324
--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.4875603410
1 O     0.0000000000     0.0000000000    -0.4875603410
----------------------------------------------


In [26]:
ec=scf.RHF(gto.M(atom="C 0 0 0")).scf()
eo=scf.RHF(gto.M(atom="O 0 0 0")).scf()

converged SCF energy = -37.0895866394329
converged SCF energy = -73.6618168979007


In [29]:
bde=ec+eo-e_co

In [30]:
g,h,bde

(-0.4875603410073399, 3.1323209757043315, 0.42766073999034404)

In [8]:
De=227/627
Z=g**2/(2*De)/h

$$ t^3 -2t^2+t =Z(2t-1) $$ <br>$$
t^3-2t^2+(1-2Z)t+Z=0
$$
To be solved for t>0

In [9]:
np.roots([1,-2,1-2*Z,+Z])

array([ 1.36442204,  0.73945964, -0.10388168])

In [10]:
t=float(np.roots([1,-2,1-2*Z,+Z])[0])
t

1.3644220421049187

$$ a=\frac{\tilde g}{t(1-t)}$$

In [11]:
a=g/(2*De)/(t*(1-t))
a

1.354212118535271

$$ t=e^{-a(r-r_e)}
$$

In [12]:
dX=np.log(t)/a

In [13]:
R0+dX

2.17945513656408

In [14]:
R0-g/h

2.105654655059019

In [22]:
def mpa(g,h,R0,De): # Morse Potential Approximation
    Z=g**2/(2*De)/h
    for i in range(3):
        t=float(np.roots([1,-2,1-2*Z,+Z])[i])
        print(np.roots([1,-2,1-2*Z,+Z]))
        a=g/(2*De)/(t*(1-t))
        print(a)
        dX=np.log(t)/a
        if a>0 and t>0:
            return R0+dX
    return -1

In [37]:
mpa(g,h,R0,bde)

[ 1.33300725  0.75513861 -0.08814586]
1.2841412495833624


2.173836340804822